In [80]:
import requests
import pandas as pd
from datetime import datetime, timedelta
import pytz
pd.reset_option('display.max_rows')
#pd.set_option('display.max_rows', None)

In [95]:
def shift_col(team, col_name):
    next_col = team[col_name].shift(-1)
    return next_col

def add_col(df, col_name):
    return df.groupby("Teams", group_keys=False).apply(lambda x: shift_col(x, col_name))

In [96]:
folder_path = "/Users/benjamincheng/Documents/GitHub/Sports-Betting/nba_api/data/teams_stats/updates/2024_04_04.csv"
df = pd.read_csv(folder_path, index_col=0)
df = df.reset_index(drop=True)

In [97]:
date_prediction = '2024-04-04'


df = df[df['Date'] == '2024-04-03']
df['date_next'] = add_col(df, 'Date')
df['date_next'].fillna(date_prediction, inplace=True)
dates = df['date_next'].unique()

/var/folders/99/zv3w4flx1598dxlknc099kfr0000gn/T/ipykernel_86963/138724945.py:6: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  return df.groupby("Teams", group_keys=False).apply(lambda x: shift_col(x, col_name))
/var/folders/99/zv3w4flx1598dxlknc099kfr0000gn/T/ipykernel_86963/3589403161.py:6: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or

In [99]:
mapping = {'Brooklyn Nets': 'BRK', 'Golden State Warriors': 'GSW', 'Los Angeles Lakers': 'LAL',
       'Milwaukee Bucks': 'MIL', 'Boston Celtics': 'BOS', 'Charlotte Hornets': 'CHO',
       'Chicago Bulls': 'CHI', 'Cleveland Cavaliers': 'CLE', 'Denver Nuggets': 'DEN',
       'Detroit Pistons': 'DET', 'Houston Rockets': 'HOU', 'Indiana Pacers': 'IND',
       'Memphis Grizzlies': 'MEM', 'Minnesota Timberwolves': 'MIN',
       'New Orleans Pelicans': 'NOP', 'New York Knicks': 'NYK', 'Oklahoma City Thunder': 'OKC',
       'Orlando Magic': 'ORL', 'Philadelphia 76ers': 'PHI', 'Phoenix Suns': 'PHO',
       'Portland Trail Blazers': 'POR', 'Sacramento Kings': 'SAC', 'San Antonio Spurs': 'SAS',
       'Toronto Raptors': 'TOR', 'Utah Jazz': 'UTA', 'Washington Wizards': 'WAS',
       'Atlanta Hawks': 'ATL', 'Dallas Mavericks': 'DAL', 'Los Angeles Clippers': 'LAC', 'Miami Heat': 'MIA'}

In [100]:
# Define time zones
dates_utc = []
est_timezone = pytz.timezone('America/New_York')  # Eastern Standard Time (EST)
utc_timezone = pytz.timezone('UTC')

threshold_value = pd.to_datetime('2020-06-27T03:55:00Z')

full = []
for date in dates:
    date = date[:10]
    # Create a datetime object for the given time in EST
    est_time = datetime.strptime(f'{date} 4:00 PM', '%Y-%m-%d %I:%M %p')
    est_time = est_timezone.localize(est_time)

    # Convert EST time to UTC
    utc_time = est_time.astimezone(utc_timezone)
    if utc_time >= threshold_value:
        final_time = utc_time.strftime('%Y-%m-%dT%H:%M:%SZ')
        dates_utc.append(final_time)

In [101]:
KEY = 'dc34ee156925bf51d4a4c33c87d440fc'
SPORT = 'basketball_nba'
REGION = 'us'
MARKET = 'h2h'
FORMAT = 'decimal'
full = []
# adjust number of dates 
DATES = dates_utc
for DATE in DATES:
    response = requests.get(f'https://api.the-odds-api.com/v4/historical/sports/{SPORT}/odds',
        params={
            'api_key': KEY,
            'regions': REGION,
            'markets': MARKET,
            'oddsFormat': FORMAT,
            'date': DATE,
        }
    ).json()
    for i in range(len(response['data'])):
        # [Timestamp, good team, evil team, fanduel odds good team wins, draftkings odds, caesars odds]
        #best_odds = [[None,None,None,None,float('-inf')] for _ in range(2)]
        bookmakers = response['data'][i]['bookmakers']
        for index, data in enumerate(bookmakers):
            best_odds = [[None,None,None,None,float('-inf')] for _ in range(2)]
            if response['data'][i]['bookmakers'][index]['title'] in ["FanDuel", "BetMGM", "DraftKings", ""]:
                for outcome in range(2):
                    odds = response['data'][i]['bookmakers'][index]['markets'][0]['outcomes'][outcome]['price']
                    current_odds = best_odds[outcome][4]
                    best_odds[outcome][0] = response['timestamp']
                    best_odds[outcome][1] = response['data'][i]['id']
                    best_odds[outcome][2] = response['data'][i]['bookmakers'][index]['title']
                    best_odds[outcome][3] = response['data'][i]['bookmakers'][index]['markets'][0]['outcomes'][outcome]['name']
                    best_odds[outcome][4] = odds
                full.extend(best_odds)

In [102]:
timestamp = []
Id = []
sportsbook = []
team = []
odds = []
for i in range(len(full)):
    timestamp.append(full[i][0])
    Id.append(full[i][1])
    sportsbook.append(full[i][2])
    team.append(full[i][3])
    odds.append(full[i][4])
df = pd.DataFrame({"Timestamp": timestamp,"Id": Id, "Sportsbook": sportsbook, "Team": team, "Odds": odds})

In [103]:
df

,Timestamp,Id,Sportsbook,Team,Odds
0,2024-04-04T19:55:39Z,4fa62836feeb88c78b9d54c8274d6add,DraftKings,Atlanta Hawks,5.90
1,2024-04-04T19:55:39Z,4fa62836feeb88c78b9d54c8274d6add,DraftKings,Dallas Mavericks,1.15
2,2024-04-04T19:55:39Z,4fa62836feeb88c78b9d54c8274d6add,FanDuel,Atlanta Hawks,6.00
3,2024-04-04T19:55:39Z,4fa62836feeb88c78b9d54c8274d6add,FanDuel,Dallas Mavericks,1.14
4,2024-04-04T19:55:39Z,4fa62836feeb88c78b9d54c8274d6add,BetMGM,Atlanta Hawks,6.00
5,2024-04-04T19:55:39Z,4fa62836feeb88c78b9d54c8274d6add,BetMGM,Dallas Mavericks,1.14
6,2024-04-04T19:55:39Z,e3ca97338d3a63129be2cf0f6111edb9,DraftKings,Miami Heat,1.77
7,2024-04-04T19:55:39Z,e3ca97338d3a63129be2cf0f6111edb9,DraftKings,Philadelphia 76ers,2.10
8,2024-04-04T19:55:39Z,e3ca97338d3a63129be2cf0f6111edb9,FanDuel,Miami Heat,1.77
9,2024-04-04T19:55:39Z,e3ca97338d3a63129be2cf0f6111edb9,FanDuel,Philadelphia 76ers,2.10


In [104]:
grouped = df.groupby('Id')
group_sizes = grouped.size()

# Filter groups with less than 6 rows
ids_with_less_than_six_rows = group_sizes[group_sizes < 6].index.tolist()

print("Ids with less than 6 rows:")

ids_with_less_than_six_rows

Ids with less than 6 rows:


[]

In [105]:
miss = []
for Id in ids_with_less_than_six_rows:
    group = df[df['Id'] == Id]
    sportsbooks = ["FanDuel", "BetMGM", "DraftKings"]
    
    # check which sportsbook is missing
    present_sportsbooks = group['Sportsbook'].unique()
    missing_sportsbooks = [sb for sb in sportsbooks if sb not in present_sportsbooks][0]

    teams = group['Team'].unique()
    for team in teams:
        missing = [None, None, None, None, float('-inf')]
        # calculate average odds 
        missing[0] = group['Timestamp'].iloc[0]
        missing[1] = Id
        missing[2] = missing_sportsbooks
        missing[3] = team
        missing[4] = round(group[group['Team'] == team]['Odds'].mean(), 2)
        miss.append(missing)
missing_df = pd.DataFrame(miss, columns=['Timestamp', 'Id', 'Sportsbook', 'Team', 'Odds'])

In [106]:
concat = pd.concat([df, missing_df], ignore_index=True)
concat = concat.sort_values(by=['Timestamp', 'Id'])
concat = concat.reset_index(drop=True)
full = concat.merge(concat,
               left_on=['Timestamp','Id', 'Sportsbook'],
               right_on = ['Timestamp','Id', 'Sportsbook'])
full = full.drop(full[full['Team_x'] == full['Team_y']].index)
full = full.reset_index(drop=True)

/var/folders/99/zv3w4flx1598dxlknc099kfr0000gn/T/ipykernel_86963/1165239269.py:1: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  concat = pd.concat([df, missing_df], ignore_index=True)


In [107]:
full

,Timestamp,Id,Sportsbook,Team_x,Odds_x,Team_y,Odds_y
0,2024-04-04T19:55:39Z,4fa62836feeb88c78b9d54c8274d6add,DraftKings,Atlanta Hawks,5.90,Dallas Mavericks,1.15
1,2024-04-04T19:55:39Z,4fa62836feeb88c78b9d54c8274d6add,DraftKings,Dallas Mavericks,1.15,Atlanta Hawks,5.90
2,2024-04-04T19:55:39Z,4fa62836feeb88c78b9d54c8274d6add,FanDuel,Atlanta Hawks,6.00,Dallas Mavericks,1.14
3,2024-04-04T19:55:39Z,4fa62836feeb88c78b9d54c8274d6add,FanDuel,Dallas Mavericks,1.14,Atlanta Hawks,6.00
4,2024-04-04T19:55:39Z,4fa62836feeb88c78b9d54c8274d6add,BetMGM,Atlanta Hawks,6.00,Dallas Mavericks,1.14
5,2024-04-04T19:55:39Z,4fa62836feeb88c78b9d54c8274d6add,BetMGM,Dallas Mavericks,1.14,Atlanta Hawks,6.00
6,2024-04-04T19:55:39Z,54e9789714fd7f45e04c1a7b3dececd5,DraftKings,Golden State Warriors,1.59,Houston Rockets,2.42
7,2024-04-04T19:55:39Z,54e9789714fd7f45e04c1a7b3dececd5,DraftKings,Houston Rockets,2.42,Golden State Warriors,1.59
8,2024-04-04T19:55:39Z,54e9789714fd7f45e04c1a7b3dececd5,FanDuel,Golden State Warriors,1.60,Houston Rockets,2.40
9,2024-04-04T19:55:39Z,54e9789714fd7f45e04c1a7b3dececd5,FanDuel,Houston Rockets,2.40,Golden State Warriors,1.60


In [108]:
full['Timestamp'] = pd.to_datetime(full['Timestamp'])

utc_timezone = pytz.utc
est_timezone = pytz.timezone('US/Eastern')

full['Timestamp'] = full['Timestamp'].dt.tz_convert(est_timezone)
full['Timestamp'] = full['Timestamp'].dt.strftime('%Y-%m-%d')

In [109]:
full['Team_x'] = full['Team_x'].map(mapping)
full['Team_y'] = full['Team_y'].map(mapping)

In [110]:
full

,Timestamp,Id,Sportsbook,Team_x,Odds_x,Team_y,Odds_y
0,2024-04-04,4fa62836feeb88c78b9d54c8274d6add,DraftKings,ATL,5.90,DAL,1.15
1,2024-04-04,4fa62836feeb88c78b9d54c8274d6add,DraftKings,DAL,1.15,ATL,5.90
2,2024-04-04,4fa62836feeb88c78b9d54c8274d6add,FanDuel,ATL,6.00,DAL,1.14
3,2024-04-04,4fa62836feeb88c78b9d54c8274d6add,FanDuel,DAL,1.14,ATL,6.00
4,2024-04-04,4fa62836feeb88c78b9d54c8274d6add,BetMGM,ATL,6.00,DAL,1.14
5,2024-04-04,4fa62836feeb88c78b9d54c8274d6add,BetMGM,DAL,1.14,ATL,6.00
6,2024-04-04,54e9789714fd7f45e04c1a7b3dececd5,DraftKings,GSW,1.59,HOU,2.42
7,2024-04-04,54e9789714fd7f45e04c1a7b3dececd5,DraftKings,HOU,2.42,GSW,1.59
8,2024-04-04,54e9789714fd7f45e04c1a7b3dececd5,FanDuel,GSW,1.60,HOU,2.40
9,2024-04-04,54e9789714fd7f45e04c1a7b3dececd5,FanDuel,HOU,2.40,GSW,1.60


In [111]:
fanduel = full[full['Sportsbook'] == 'FanDuel']
columns = fanduel.columns
columns = ['Timestamp', 'Id', 'Sportsbook', 'Team_x', 'Fanduel_odds_x','Team_y', 'Fanduel_odds_y']
fanduel.columns = columns
fanduel = fanduel.drop('Sportsbook', axis=1)

draftkings = full[full['Sportsbook'] == 'DraftKings']
columns = draftkings.columns
columns = ['Timestamp', 'Id', 'Sportsbook', 'Team_x', 'Draftkings_odds_x','Team_y', 'Draftkings_odds_y']
draftkings.columns = columns
draftkings = draftkings.drop('Sportsbook', axis=1)

betMGM = full[full['Sportsbook'] == 'BetMGM']
columns = betMGM.columns
columns = ['Timestamp', 'Id', 'Sportsbook', 'Team_x', 'BetMGM_odds_x','Team_y', 'BetMGM_odds_y']
betMGM.columns = columns
betMGM = betMGM.drop('Sportsbook', axis=1)

In [112]:
merged_fd_dk = pd.merge(fanduel, draftkings, on=['Timestamp', 'Id', 'Team_x', 'Team_y'], how='left')

# Merge the result with betMGM DataFrame
final = pd.merge(merged_fd_dk, betMGM, on=['Timestamp', 'Id', 'Team_x', 'Team_y'], how='left')

In [113]:
columns = final.columns
columns = ['Timestamp', 'Id', 'Teams_x', 'Fanduel_odds_x','Teams_y', 'Fanduel_odds_y', 'Draftkings_odds_x', 'Draftkings_odds_y', 'BetMGM_odds_x', 'BetMGM_odds_y']
final.columns = columns
final = final.drop('Id', axis=1)

order = ['Timestamp', 'Teams_x', 'Fanduel_odds_x','Draftkings_odds_x', 'BetMGM_odds_x', 'Teams_y', 'Fanduel_odds_y','Draftkings_odds_y', 'BetMGM_odds_y']
final = final[order]

In [114]:
final

,Timestamp,Teams_x,Fanduel_odds_x,Draftkings_odds_x,BetMGM_odds_x,Teams_y,Fanduel_odds_y,Draftkings_odds_y,BetMGM_odds_y
0,2024-04-04,ATL,6.00,5.90,6.00,DAL,1.14,1.15,1.14
1,2024-04-04,DAL,1.14,1.15,1.14,ATL,6.00,5.90,6.00
2,2024-04-04,GSW,1.60,1.59,1.61,HOU,2.40,2.42,2.40
3,2024-04-04,HOU,2.40,2.42,2.40,GSW,1.60,1.59,1.61
4,2024-04-04,DEN,1.60,1.57,1.57,LAC,2.42,2.45,2.45
5,2024-04-04,LAC,2.42,2.45,2.45,DEN,1.60,1.57,1.57
6,2024-04-04,NYK,1.65,1.65,1.62,SAC,2.30,2.30,2.35
7,2024-04-04,SAC,2.30,2.30,2.35,NYK,1.65,1.65,1.62
8,2024-04-04,MIA,1.77,1.77,1.74,PHI,2.10,2.10,2.10
9,2024-04-04,PHI,2.10,2.10,2.10,MIA,1.77,1.77,1.74


In [115]:
save_path = "/Users/benjamincheng/Documents/GitHub/Sports-Betting/data/odds_data/"
file_name = "2021_01_20_onward.csv"
file_path = save_path + file_name
whole = pd.read_csv(file_path, index_col=0)

In [116]:
complete = pd.concat([whole, final], axis=0)

In [117]:
save_path = "/Users/benjamincheng/Documents/GitHub/Sports-Betting/data/odds_data/"
file_name = "2021_01_20_onward.csv"
file_path = save_path + file_name
complete.to_csv(file_path)